In [7]:
import pandas as pd 
import folium
import geopy

In [8]:
import requests 
import random

In [9]:
from geopy.geocoders import Nominatim  #  use to convert adderace to the lattitude and longitudes
from IPython.display import Image
from IPython.display import HTML

In [10]:
from pandas.io.json import json_normalize # this module is used to convet the json into the dataframe

In [14]:
# foursquar API credentials

CLIENT_ID = 'GOZTODACKFPTYEFWH4C4EJQ40O04EOS5PZTAKK0XLGAB4EH4' # your Foursquare ID
CLIENT_SECRET = '5ZFJT4HCZPPUE4CVA2QIBA4RGKHWZFF1R4NQGVBKOUOGGSV3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 1000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GOZTODACKFPTYEFWH4C4EJQ40O04EOS5PZTAKK0XLGAB4EH4
CLIENT_SECRET:5ZFJT4HCZPPUE4CVA2QIBA4RGKHWZFF1R4NQGVBKOUOGGSV3


In [15]:
address = '102 North End Ave, New York, NY'

geolocator=Nominatim(user_agent='foursquare_api')
location = geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

40.7149555 -74.0153365


In [16]:
search_query='Indian'
radius=35000
print(search_query+'-----OK')

Indian-----OK


In [17]:

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOZTODACKFPTYEFWH4C4EJQ40O04EOS5PZTAKK0XLGAB4EH4&client_secret=5ZFJT4HCZPPUE4CVA2QIBA4RGKHWZFF1R4NQGVBKOUOGGSV3&ll=40.7149555,-74.0153365&v=20180604&query=Indian&radius=35000&limit=1000'

In [18]:
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5cdb1fa6db04f559dba485fe'},
 'response': {'venues': [{'id': '5072fe2eb0ed0fcce2497f30',
    'name': 'Indian Biryani Delights (Cart)',
    'location': {'address': 'Water St',
     'crossStreet': 'btwn Maiden Lane & Pine St',
     'lat': 40.71441853663672,
     'lng': -74.01157314448908,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71441853663672,
       'lng': -74.01157314448908}],
     'distance': 323,
     'postalCode': '10005',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['Water St (btwn Maiden Lane & Pine St)',
      'New York, NY 10005',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1cb941735',
      'name': 'Food Truck',
      'pluralName': 'Food Trucks',
      'shortName': 'Food Truck',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/streetfood_',
       'suffix': '.png'},
      'primary': True}],
    'referra

In [19]:
venue=result['response']['venues']
venue_dataframe=json_normalize(venue)

In [20]:
print(venue_dataframe.head(10))

                                          categories delivery.id  \
0  [{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...         NaN   
1  [{'id': '4bf58dd8d48988d181941735', 'name': 'M...         NaN   
2  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...      275521   
3  [{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...         NaN   
4  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...         NaN   
5  [{'id': '52f2ab2ebcbc57f1066b8b1b', 'name': 'S...         NaN   
6  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...       79935   
7  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...      266126   
8  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...      333939   
9  [{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...         NaN   

                delivery.provider.icon.name  \
0                                       NaN   
1                                       NaN   
2  /delivery_provider_seamless_20180129.png   
3                                       NaN   
4               

In [21]:
filtered_columns = ['name', 'categories'] + [col for col in venue_dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = venue_dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Indian Biryani Delights (Cart),Food Truck,Water St,US,New York,United States,btwn Maiden Lane & Pine St,323,"[Water St (btwn Maiden Lane & Pine St), New Yo...","[{'label': 'display', 'lat': 40.71441853663672...",40.714419,-74.011573,NaN,10005,NY,5072fe2eb0ed0fcce2497f30
1,National Museum of the American Indian,Museum,1 Bowling Grn,US,New York,United States,btwn Broadway & State St,1193,"[1 Bowling Grn (btwn Broadway & State St), New...","[{'label': 'display', 'lat': 40.7043035374255,...",40.704304,-74.013759,NaN,10004,NY,45719c4bf964a520693e1fe3
2,A Saffron Thread Fresh Indian,Indian Restaurant,98 Chambers St,US,New York,United States,at Church St,645,"[98 Chambers St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71482256030567...",40.714823,-74.007689,NaN,10007,NY,510aad1ae4b0bfb2583169b9
3,Indian king Biriyani House,Food Truck,140 Broadway,US,New York,United States,NaN,1627,"[140 Broadway, New York, NY 10005, United States]","[{'label': 'display', 'lat': 40.72821194986679...",40.728212,-74.007189,Tribeca,10005,NY,573f435b498e03b8bb8582c7
4,Panna II Garden Indian Restaurant,Indian Restaurant,93 1st Ave,US,New York,United States,btwn 5th & 6th St,2756,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...","[{'label': 'display', 'lat': 40.72627299004598...",40.726273,-73.986273,NaN,10003,NY,4116be80f964a520f90b1fe3
5,National Museum of the American Indian Museum ...,Souvenir Shop,1 Bowling Grn,US,New York,United States,NaN,1146,"[1 Bowling Grn, New York, NY 10004, United Sta...","[{'label': 'display', 'lat': 40.70477695730307...",40.704777,-74.013249,NaN,10004,NY,54a4690e498e43a77f503eba
6,Ruchi Indian Cuisine,Indian Restaurant,120 Cedar St,US,New York,United States,btwn Greenwich St. & Trinity Pl.,590,[120 Cedar St (btwn Greenwich St. & Trinity Pl...,"[{'label': 'display', 'lat': 40.70996196046026...",40.709962,-74.012969,NaN,10006,NY,4b12cb6ff964a5204a8e23e3
7,Asya Indian Restaurant,Indian Restaurant,46 Henry St,US,Brooklyn,United States,NaN,2600,"[46 Henry St, Brooklyn, NY 11201, United States]","[{'label': 'display', 'lat': 40.699607, 'lng':...",40.699607,-73.992110,NaN,11201,NY,506a2625498e3c434686abbc
8,Aahar Indian Cuisine,Indian Restaurant,10 Murray St,US,New York,United States,Broadway,646,"[10 Murray St (Broadway), New York, NY 10007, ...","[{'label': 'display', 'lat': 40.71330656515891...",40.713307,-74.007994,NaN,10007,NY,575dea4c498e2739e43a27e2
9,Indian Biryani House,Food Truck,NaN,US,New York,United States,NaN,736,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.7093203968901,...",40.709320,-74.010758,NaN,NaN,NY,553fb844498e6cc1b6da5ce4


In [22]:
dataframe_filtered.name

0                        Indian Biryani Delights (Cart)
1                National Museum of the American Indian
2                         A Saffron Thread Fresh Indian
3                            Indian king Biriyani House
4                     Panna II Garden Indian Restaurant
5     National Museum of the American Indian Museum ...
6                                  Ruchi Indian Cuisine
7                                Asya Indian Restaurant
8                                  Aahar Indian Cuisine
9                                  Indian Biryani House
10                                        Swagat Indian
11                                 Kiran Indian Cuisine
12                   2 Darbar Grill Fine Indian Cuisine
13                              Sapphire Indian Cuisine
14                           Bombay's Indian Restaurant
15                                     Indian Road Café
16            Indian Beautiful Art (IBA) Crafts PVT Ltd
17                                       Indian 

In [23]:
dataframe_newyork=dataframe_filtered[dataframe_filtered.city=="New York"]

In [24]:
dataframe_newyork

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Indian Biryani Delights (Cart),Food Truck,Water St,US,New York,United States,btwn Maiden Lane & Pine St,323,"[Water St (btwn Maiden Lane & Pine St), New Yo...","[{'label': 'display', 'lat': 40.71441853663672...",40.714419,-74.011573,NaN,10005,NY,5072fe2eb0ed0fcce2497f30
1,National Museum of the American Indian,Museum,1 Bowling Grn,US,New York,United States,btwn Broadway & State St,1193,"[1 Bowling Grn (btwn Broadway & State St), New...","[{'label': 'display', 'lat': 40.7043035374255,...",40.704304,-74.013759,NaN,10004,NY,45719c4bf964a520693e1fe3
2,A Saffron Thread Fresh Indian,Indian Restaurant,98 Chambers St,US,New York,United States,at Church St,645,"[98 Chambers St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71482256030567...",40.714823,-74.007689,NaN,10007,NY,510aad1ae4b0bfb2583169b9
3,Indian king Biriyani House,Food Truck,140 Broadway,US,New York,United States,NaN,1627,"[140 Broadway, New York, NY 10005, United States]","[{'label': 'display', 'lat': 40.72821194986679...",40.728212,-74.007189,Tribeca,10005,NY,573f435b498e03b8bb8582c7
4,Panna II Garden Indian Restaurant,Indian Restaurant,93 1st Ave,US,New York,United States,btwn 5th & 6th St,2756,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...","[{'label': 'display', 'lat': 40.72627299004598...",40.726273,-73.986273,NaN,10003,NY,4116be80f964a520f90b1fe3
5,National Museum of the American Indian Museum ...,Souvenir Shop,1 Bowling Grn,US,New York,United States,NaN,1146,"[1 Bowling Grn, New York, NY 10004, United Sta...","[{'label': 'display', 'lat': 40.70477695730307...",40.704777,-74.013249,NaN,10004,NY,54a4690e498e43a77f503eba
6,Ruchi Indian Cuisine,Indian Restaurant,120 Cedar St,US,New York,United States,btwn Greenwich St. & Trinity Pl.,590,[120 Cedar St (btwn Greenwich St. & Trinity Pl...,"[{'label': 'display', 'lat': 40.70996196046026...",40.709962,-74.012969,NaN,10006,NY,4b12cb6ff964a5204a8e23e3
8,Aahar Indian Cuisine,Indian Restaurant,10 Murray St,US,New York,United States,Broadway,646,"[10 Murray St (Broadway), New York, NY 10007, ...","[{'label': 'display', 'lat': 40.71330656515891...",40.713307,-74.007994,NaN,10007,NY,575dea4c498e2739e43a27e2
9,Indian Biryani House,Food Truck,NaN,US,New York,United States,NaN,736,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.7093203968901,...",40.709320,-74.010758,NaN,NaN,NY,553fb844498e6cc1b6da5ce4
10,Swagat Indian,Indian Restaurant,205 W 29th St,US,New York,United States,7th Ave,4133,"[205 W 29th St (7th Ave), New York, NY 10001, ...","[{'label': 'display', 'lat': 40.74804551427938...",40.748046,-73.993108,NaN,10001,NY,4ae015bff964a5202b7e21e3


In [25]:
len(dataframe_newyork)

37

In [26]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_newyork.lat, dataframe_newyork.lng, dataframe_newyork.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [27]:
print(" Total Number of Desi Restaurent in the New York City : ", len(dataframe_newyork))

 Total Number of Indian Restaurent in the New York City :  37


In [28]:
dataframe_newyork.groupby("postalCode").size().reset_index(name="Counts")

,postalCode,Counts
0,10001,2
1,10003,2
2,10004,4
3,10005,2
4,10006,1
5,10007,7
6,10011,1
7,10013,2
8,10016,1
9,10017,1


In [29]:
Newyork_counts= dataframe_newyork.groupby("postalCode").count()

In [30]:
dataframe_newyork

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Indian Biryani Delights (Cart),Food Truck,Water St,US,New York,United States,btwn Maiden Lane & Pine St,323,"[Water St (btwn Maiden Lane & Pine St), New Yo...","[{'label': 'display', 'lat': 40.71441853663672...",40.714419,-74.011573,NaN,10005,NY,5072fe2eb0ed0fcce2497f30
1,National Museum of the American Indian,Museum,1 Bowling Grn,US,New York,United States,btwn Broadway & State St,1193,"[1 Bowling Grn (btwn Broadway & State St), New...","[{'label': 'display', 'lat': 40.7043035374255,...",40.704304,-74.013759,NaN,10004,NY,45719c4bf964a520693e1fe3
2,A Saffron Thread Fresh Indian,Indian Restaurant,98 Chambers St,US,New York,United States,at Church St,645,"[98 Chambers St (at Church St), New York, NY 1...","[{'label': 'display', 'lat': 40.71482256030567...",40.714823,-74.007689,NaN,10007,NY,510aad1ae4b0bfb2583169b9
3,Indian king Biriyani House,Food Truck,140 Broadway,US,New York,United States,NaN,1627,"[140 Broadway, New York, NY 10005, United States]","[{'label': 'display', 'lat': 40.72821194986679...",40.728212,-74.007189,Tribeca,10005,NY,573f435b498e03b8bb8582c7
4,Panna II Garden Indian Restaurant,Indian Restaurant,93 1st Ave,US,New York,United States,btwn 5th & 6th St,2756,"[93 1st Ave (btwn 5th & 6th St), New York, NY ...","[{'label': 'display', 'lat': 40.72627299004598...",40.726273,-73.986273,NaN,10003,NY,4116be80f964a520f90b1fe3
5,National Museum of the American Indian Museum ...,Souvenir Shop,1 Bowling Grn,US,New York,United States,NaN,1146,"[1 Bowling Grn, New York, NY 10004, United Sta...","[{'label': 'display', 'lat': 40.70477695730307...",40.704777,-74.013249,NaN,10004,NY,54a4690e498e43a77f503eba
6,Ruchi Indian Cuisine,Indian Restaurant,120 Cedar St,US,New York,United States,btwn Greenwich St. & Trinity Pl.,590,[120 Cedar St (btwn Greenwich St. & Trinity Pl...,"[{'label': 'display', 'lat': 40.70996196046026...",40.709962,-74.012969,NaN,10006,NY,4b12cb6ff964a5204a8e23e3
8,Aahar Indian Cuisine,Indian Restaurant,10 Murray St,US,New York,United States,Broadway,646,"[10 Murray St (Broadway), New York, NY 10007, ...","[{'label': 'display', 'lat': 40.71330656515891...",40.713307,-74.007994,NaN,10007,NY,575dea4c498e2739e43a27e2
9,Indian Biryani House,Food Truck,NaN,US,New York,United States,NaN,736,"[New York, NY, United States]","[{'label': 'display', 'lat': 40.7093203968901,...",40.709320,-74.010758,NaN,NaN,NY,553fb844498e6cc1b6da5ce4
10,Swagat Indian,Indian Restaurant,205 W 29th St,US,New York,United States,7th Ave,4133,"[205 W 29th St (7th Ave), New York, NY 10001, ...","[{'label': 'display', 'lat': 40.74804551427938...",40.748046,-73.993108,NaN,10001,NY,4ae015bff964a5202b7e21e3


In [31]:
from sklearn.cluster import KMeans
kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Newyork_counts)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 4, 3, 0, 1, 0, 3, 0, 0], dtype=int32)

In [ ]:
import numpy as np
import matplotlib.cm as cm
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dataframe_newyork['lat'], dataframe_newyork['lng'], dataframe_newyork['postalCode'], dataframe_newyork['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [34]:
address1 = '1667 K Street NW,Washington,DC'

geolocator=Nominatim(user_agent='foursquare_api')
location = geolocator.geocode(address1)
latitude=location.latitude
longitude=location.longitude
print(latitude,longitude)

38.902781 -77.038203


In [35]:
search_query='Indian'
radius=35000
print(search_query+'-----OK')

Indian-----OK


In [36]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=GOZTODACKFPTYEFWH4C4EJQ40O04EOS5PZTAKK0XLGAB4EH4&client_secret=5ZFJT4HCZPPUE4CVA2QIBA4RGKHWZFF1R4NQGVBKOUOGGSV3&ll=38.902781,-77.038203&v=20180604&query=Indian&radius=35000&limit=1000'

In [37]:
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5cdb214e1ed2196dbeed0de1'},
 'response': {'venues': [{'id': '49e87228f964a5204c651fe3',
    'name': 'National Museum of the American Indian',
    'location': {'address': '950 Independence Ave SW',
     'crossStreet': '4th St. SW',
     'lat': 38.888141,
     'lng': -77.016476,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.888141,
       'lng': -77.016476}],
     'distance': 2489,
     'postalCode': '20560',
     'cc': 'US',
     'city': 'Washington',
     'state': 'D.C.',
     'country': 'United States',
     'formattedAddress': ['950 Independence Ave SW (4th St. SW)',
      'Washington, D.C. 20560',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d190941735',
      'name': 'History Museum',
      'pluralName': 'History Museums',
      'shortName': 'History Museum',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_history_',
       'suffix': '.png'},
      'primary': True}],

In [38]:
venue=result['response']['venues']
venue_dataframe=json_normalize(venue)

In [39]:
print(venue_dataframe.head(10))

                                          categories delivery.id  \
0  [{'id': '4bf58dd8d48988d190941735', 'name': 'H...         NaN   
1  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...         NaN   
2  [{'id': '4bf58dd8d48988d126941735', 'name': 'G...         NaN   
3  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...      285455   
4  [{'id': '4bf58dd8d48988d124941735', 'name': 'O...         NaN   
5  [{'id': '4bf58dd8d48988d10f941735', 'name': 'I...       68888   
6  [{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...         NaN   
7  [{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...         NaN   
8  [{'id': '4bf58dd8d48988d181941735', 'name': 'M...         NaN   
9  [{'id': '4bf58dd8d48988d190941735', 'name': 'H...         NaN   

               delivery.provider.icon.name  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3  /delivery_provider_grubhub_20180129.png   
4                    

In [40]:
filtered_columns = ['name', 'categories'] + [col for col in venue_dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = venue_dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,National Museum of the American Indian,History Museum,950 Independence Ave SW,US,Washington,United States,4th St. SW,2489,"[950 Independence Ave SW (4th St. SW), Washing...","[{'label': 'display', 'lat': 38.888141, 'lng':...",38.888141,-77.016476,20560,D.C.,49e87228f964a5204c651fe3
1,Indian Delight,Indian Restaurant,1825 I St NW,US,Washington,United States,at 18 St NW,354,"[1825 I St NW (at 18 St NW), Washington, D.C. ...","[{'label': 'display', 'lat': 38.90141933084688...",38.901419,-77.041905,20006,D.C.,4b8acd7bf964a5200d8232e3
2,"Indian Treaty Room, EEOB",Government Building,NaN,US,Washington,United States,NaN,419,"[Washington, D.C., United States]","[{'label': 'display', 'lat': 38.89913080229488...",38.899131,-77.039414,NaN,D.C.,4f6a29f7e4b086028beabe63
3,Rasoi Indian Kitchen,Indian Restaurant,810 K St NW,US,Washington,United States,18th,1314,"[810 K St NW (18th), Washington, D.C. 20001, U...","[{'label': 'display', 'lat': 38.902108, 'lng':...",38.902108,-77.023053,20001,D.C.,4c7448e866be6dcb3f51bd0f
4,Bureau of Indian Education,Office,1849 C St.,US,Washingt,United States,19th St.,1039,"[1849 C St. (19th St.), Washingt, D.C. 20240, ...","[{'label': 'display', 'lat': 38.89479768080878...",38.894798,-77.044429,20240,D.C.,501ab5d7e4b001692563b059
5,Rajaji Fine Indian Cuisine,Indian Restaurant,2603 Connecticut Ave NW,US,Washington,United States,Calvert,2609,"[2603 Connecticut Ave NW (Calvert), Washington...","[{'label': 'display', 'lat': 38.92375473826234...",38.923755,-77.051662,20008,D.C.,4c110fdc3ce120a12e0c091c
6,Mint Indian Cuisine,Food Truck,NaN,US,Washington,United States,NaN,136,"[Washington, D.C., United States]","[{'label': 'display', 'lat': 38.90237911040122...",38.902379,-77.039690,NaN,D.C.,530f76cc11d26025ecb9bc68
7,Zyaka Indian Cuisine,Food Truck,NaN,US,Washington,United States,NaN,157,"[Washington, D.C., United States]","[{'label': 'display', 'lat': 38.90168967299778...",38.901690,-77.039358,NaN,D.C.,58a34557a36ecd4f52732198
8,Potomac Atrium - Smithsonian National Museum o...,Museum,Independence Avenue SW,US,Washington,United States,"4th Street, Maryland Avenue, Jefferson Drive SW",1071,"[Independence Avenue SW (4th Street, Maryland ...","[{'label': 'display', 'lat': 38.89839446182729...",38.898394,-77.027195,NaN,D.C.,4d28c4f7068e8cfa0715c94c
9,Our Peoples - National Museum of the American ...,History Museum,NaN,US,Washington,United States,NaN,588,"[Washington, D.C. 20001, United States]","[{'label': 'display', 'lat': 38.90503703239283...",38.905037,-77.032060,20001,D.C.,502d2288e4b00d67ed7f89ce


In [41]:
dataframe_filtered.name

0                National Museum of the American Indian
1                                        Indian Delight
2                              Indian Treaty Room, EEOB
3                                  Rasoi Indian Kitchen
4                            Bureau of Indian Education
5                            Rajaji Fine Indian Cuisine
6                                   Mint Indian Cuisine
7                                  Zyaka Indian Cuisine
8     Potomac Atrium - Smithsonian National Museum o...
9     Our Peoples - National Museum of the American ...
10                                  Karma Modern Indian
11                    National Indian Gaming Commission
12                                 Aditi Indian Cuisine
13    Exit 3 -  MD 210 (Indian Head Hwy) / Indian He...
14                                The Indian Craft Shop
15    Always Becoming - National Museum Of The Ameri...
16                National Indian Education Association
17                             Bureau of Indian 

In [42]:

dataframe_washington=dataframe_filtered[dataframe_filtered.city=="Washington"]

In [43]:
len(dataframe_washington)

36

In [44]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_washington.lat, dataframe_washington.lng, dataframe_washington.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

In [46]:
print(" Total Number of Desi Restaurent in the New York City : ", len(dataframe_washington))

 Total Number of Desi Restaurent in the New York City :  36


In [47]:
dataframe_washington.groupby("postalCode").size().reset_index(name="Counts")

,postalCode,Counts
0,20001,4
1,20002,1
2,20003,4
3,20004,1
4,20005,2
5,20006,4
6,20007,2
7,20008,2
8,20010,1
9,20240,2


In [48]:
from sklearn.cluster import KMeans
kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Newyork_counts)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 4, 3, 0, 1, 0, 3, 0, 0], dtype=int32)

In [ ]:
import numpy as np
import matplotlib.cm as cm
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dataframe_washington['lat'], dataframe_washington['lng'], dataframe_newyork['postalCode'], dataframe_newyork['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters